In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets, transforms

import random
import numpy as np
import matplotlib.pyplot as plt

from math import ceil
from tqdm import tqdm

In [3]:
# Set manual seeds for reproducibility
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

#torch.use_deterministic_algorithms(True)

In [16]:
# Constants 
dataset_path = "../Dataset"
models_path  = "../models"
BATCH_SIZE = 64

if torch.cuda.is_available():
  _DEVICE = "cuda"
else:
  _DEVICE = "cpu"

In [5]:
# Helper functions
def count_parameters(NN):
    return sum([p.numel() for p in NN.parameters() if p.requires_grad==True])

def get_one_hot_encoder(labels):
    def ohe(target):
        oh_vec=torch.zeros(len(labels), dtype=torch.float)
        oh_vec[target] = 1.
        return oh_vec
    return ohe

In [6]:
# Dataset loader
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
dataset = datasets.ImageFolder(f'{dataset_path}/Train')
dataset = datasets.ImageFolder(f'{dataset_path}/Train', transform=transform, target_transform=get_one_hot_encoder(dataset.classes))

dataset_train, dataset_val = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), 
                                                                     ceil(len(dataset)*0.2)])

dataset_test  = datasets.ImageFolder(f'{dataset_path}/Test', transform=transform)

train_loader = data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = data.DataLoader(dataset_test,  batch_size=BATCH_SIZE, shuffle=True)
val_loader   = data.DataLoader(dataset_val,   batch_size=BATCH_SIZE, shuffle=True)

In [7]:
trainSteps = len(train_loader.dataset) // BATCH_SIZE
valSteps   = len(val_loader.dataset)  // BATCH_SIZE

In [8]:
# Load pretrained AlexNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval()

# Feezing parameters
for param in model.parameters():
    param.requires_grad = False

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/home/nzuri/HNS/HNS_Znacky_Zadanie1/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nzuri/HNS/HNS_Znacky_Zadanie1/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=9216, out_features=256, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=256, out_features=256, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=256, out_features=len(dataset.classes), bias=True)
)

In [12]:
print(f"trainable parameters: {count_parameters(model)}")

trainable parameters: 2428428


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
N_EPOCH = 100

model.to(_DEVICE)
progress_bar = tqdm(range(0, N_EPOCH))
H = {"train_loss": [],	  "val_loss": []}
for epoch in progress_bar:
    totalTrainLoss = 0
    totalValLoss = 0
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        inputs = inputs.to(_DEVICE)
        outputs = model(inputs)
        targets = targets.to(torch.float32).to(_DEVICE)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        totalTrainLoss += loss

    with torch.no_grad():
      model.eval()
      for inputs, targets in val_loader:
        inputs = inputs.to(_DEVICE)
        outputs = model(inputs)
        targets = targets.to(_DEVICE)
        totalValLoss += loss_fn(outputs, targets)

    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss   = totalValLoss / valSteps
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["val_loss"].append(avgValLoss.cpu().detach().numpy())
    progress_bar.set_description(f"Train loss: {avgTrainLoss:.6f} | Val loss: {avgValLoss:.6f}")

In [17]:
torch.save(model.state_dict(), f"{models_path}/v1.pt")

In [18]:
predictions = list()

model.eval()
with torch.no_grad():
    for inputs, targets in tqdm(test_loader):
        inputs = inputs.to(_DEVICE)
        outputs = model(inputs)
        predictions.append(
            [inputs.cpu().detach().numpy(), outputs.cpu().detach().numpy()]
        )

 93%|█████████▎| 119/128 [03:01<00:06,  1.45it/s]